In [466]:
# pip install librosa

In [467]:
'''
Please install this specific version of resampy for librosa to work without errors.
'''

'\nPlease install this specific version of resampy for librosa to work without errors.\n'

In [468]:
# pip install resampy==0.3.1

In [469]:
import soundfile
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import librosa
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings('ignore')

In [470]:
emotions ={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

### Data for binary classification


In [471]:
def load_extract_features(data_path):

    '''
    load_extract_features() is a function that is used to load all the audio files one at a time, compute their features and return the features as well as the target values.

    There are around 8-10 audio files which are corrupted. We hardcode zero values for such files in order to maintain consistency.

    ['calm', 'happy'] emotion data is categorized into 'positive' and  ['angry', 'fearful'] into 'negative'

    Returns:
    1. Features
    2. Binary Target Values
    '''
    final_features,target_emotions, binary_label = [],[], []
    count = 0
    
    for i in glob.glob(data_path + "/Actor_*/*.wav"): #Loop to read every file.
        
        name = os.path.basename(i)
        #We split the name of the file to understand the emotion associated with the file.
        split = name.split("-")
        #We know that the third identifier is associated with the emotion of the audio file. Hence, we use [2] as it represents the third identifier.
        emotion = emotions[split[2]]

        #Below is the code to categorize the emotions into two classes to make this a binary problem.
        if emotion in ['calm', 'happy']:
            binary_label.append(0)
        elif emotion in ['angry', 'fearful']:
            binary_label.append(1)
        else:
            continue
        
        with soundfile.SoundFile(i) as audio:
            waveform = audio.read(dtype="float32")
            sr = audio.samplerate
            
            #Below is the code to extract the Mel spectrogram features
            #128 is the standard for machine learning applications using Mel spectrograms
            m_feature = librosa.feature.melspectrogram(y=waveform, sr=sr, n_mels=128, fmax=sr / 2.0).T
            melspectrogram = np.mean(m_feature,axis=0)
            if melspectrogram.shape != (128,):
                melspectrogram = np.zeros(128)
            
            #Below is the code to extract the chromagram features
            stft_wave = librosa.stft(waveform)
            stft = np.abs(stft_wave)
            c_feature = librosa.feature.chroma_stft(S=stft, sr=sr).T
            chromagram = np.mean(c_feature,axis=0)
            
            #12 is the number of pitch classes
            if chromagram.shape != (12,):
                chromagram = np.zeros(12)
                
            features=np.array([])
            features=np.hstack((chromagram, melspectrogram))
        
            final_features.append(features)
            target_emotions.append(emotion)
            
            count += 1
            if count % 100 == 0:
                print("Processed Audio File Number: ", count)
    
    #We return the features and the binary target values.
    return np.array(final_features), np.array(binary_label)

In [472]:
#Please change the path below to the path of the folder saved in your computer.
# data_path = './Audio_Speech_Actors_01-24'
# X, binary_label = load_extract_features(data_path)
# print(X.shape)
# print(binary_label.shape)


In [473]:
import numpy as np
from sklearn.model_selection import train_test_split

# np.savetxt('matrix.txt', X, delimiter = ',')  
# np.savetxt('binary_label.txt',binary_label,delimiter = ',')



In [474]:
X = my_matrix = np.loadtxt(open("matrix.txt","rb"),delimiter=",",skiprows=0)
binary_label = np.loadtxt(open("binary_label.txt","rb"),delimiter=",",skiprows=0)

print(X.shape) #should be (768,140)
print(binary_label.shape) # should be (768,)

(768, 140)
(768,)


In [475]:
X = np.c_[X, np.ones(X.shape[0])]   
l = 1
X_train, X_test, y_train, y_test = train_test_split(X, binary_label, random_state=0)
print(y_test)

[0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1.]


In [476]:
def compute_cost(W, X, Y,l):
    # calculate hinge loss
    n_s = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss =  (np.sum(distances) / n_s)
    # calculate cost
    cost = l / 2 * np.dot(W, W) + hinge_loss 
    
    return cost

In [478]:
def calculate_gradient(W, X, Y,l):
  
    distance = 1 - (Y * np.dot(X, W))
    dw = np.zeros(len(W))
    for ind, d in enumerate(distance):
        if (ind == W.shape[0]):
            if max(0, d) != 0:
                di =  - ( Y[ind] * X[ind])
        else:
            if max(0, d) == 0:
                di = l*W
            else:
                di = l*W - ( Y[ind] * X[ind])
        dw += di
    dw = dw/len(Y)  # average
    return dw

In [480]:
def svm(X,Y,init_w,step,l):
    w = init_w
    cost = compute_cost(w,X,Y,l)
    for i in range(5):
        grad = calculate_gradient(w,X,Y,l)
        new_w = w - step * grad
        print(new_w)
        new_cost = compute_cost(new_w,X,Y,l)
        if (new_cost<cost):
            print("6")
            w = new_w
            cost = new_cost
    
    return w


In [482]:
def accuracy(W,X,Y):
    count = 0
    for i in range(X.shape[0]):
        yp = np.sign(np.dot(W, X[i])) #model
        if(yp == 1 and Y[i] == 1) or (yp == -1 and Y[i]== 0):
            count = count +1
    return count / Y.shape[0]